# Bilan Hydrique

Sources :

- Coefficients culturaux :
  - [ARDEPI](https://www.ardepi.fr/nos-services/vous-etes-irrigant/estimer-ses-besoins-en-eau/maraichage/)
  - [Chambre d’agriculture Nouvelle-Aquitaine](https://gironde.chambre-agriculture.fr/fileadmin/user_upload/Nouvelle-Aquitaine/100_Inst-Gironde/Documents/pdf_grandes-cultures_accompagnement-technique_mieux-irriguer/Messages_irrigation_2019/message_1/Tableau_Coefficients_Culturaux_Kc_.02.pdf)


In [1]:
import numpy as np
import pandas as pd

# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
# Altitude (m)
REF_STATION_ALTITUDE = 50.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# Météo-France API
METEOFRANCE_API = 'DPPaquetObs'

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'horaire'

# NN_NOMBRE = 20
# NN_RAYON_KM = None
NN_NOMBRE = None
NN_RAYON_KM = 35.

# Liste des variables météorologiques utilisées dans les calculs d'ETP et de bilan hydrique
LISTE_VARIABLES_METEO = ['ff', 't', 'u', 'ray_glo01', 'rr1']

# Fonction pour aggrégé chaque variable météorologique
VARIABLE_AGGREGATORS = {
    'ff': lambda x: x.mean(0),
    't': lambda x: x.mean(0),
    'u': lambda x: x.mean(0),
    'ray_glo01': lambda x: x.sum(0),
    'rr1': lambda x: x.sum(0),
    'etp': lambda x: x.sum(0),
}

In [2]:
import meteofrance

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(APPLICATION_ID, METEOFRANCE_API)

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [3]:
section = meteofrance.SECTION_LISTE_STATIONS
response = meteofrance.demande(client, section)
df_liste_stations = meteofrance.response_text_to_frame(
    client, response, index_col=client.id_station_label)

/home/atantet/.conda/envs/agri_notebooks/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portail-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
import geo

df_liste_stations_nn = geo.selection_plus_proches_voisins(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

/home/atantet/Documents/Travail/1_agri/agri-notebooks/geo.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liste_stations_nn.loc[:, 'distance'] = dist_km


In [5]:
print(f'{REF_STATION_NAME} ({REF_STATION_LATLON[0]:.6f}, {REF_STATION_LATLON[1]:.6f}) est à :')
for k, (nn_id, nn_series) in enumerate(df_liste_stations_nn.transpose().items()):
    station_name = df_liste_stations_nn.loc[nn_id, client.station_name_label]
    print(f'- {df_liste_stations_nn.loc[nn_id, 'distance']:02d} km du {k}NN {station_name} '
          f'({nn_series[client.latlon_labels[0]]:.6f}, '
          f'{nn_series[client.latlon_labels[1]]:.6f})')

La Petite Claye (48.541356, -1.615400) est à :
- 06 km du 0NN BROUALAN (48.485667, -1.640833)
- 09 km du 1NN PONTORSON (48.585667, -1.505167)
- 17 km du 2NN PLERGUER (48.524833, -1.843667)
- 24 km du 3NN FEINS  SA (48.326833, -1.596833)
- 27 km du 4NN PLESDER (48.406833, -1.924833)
- 29 km du 5NN MEZIERES-SUR-C. (48.308833, -1.439000)
- 31 km du 6NN ST OVIN (48.682500, -1.248667)
- 33 km du 7NN GRANVILLE (48.834500, -1.613667)
- 34 km du 8NN DINARD (48.584833, -2.076333)


In [6]:
df_meteo = meteofrance.compiler_donnee_des_departements(
    client, df_liste_stations_nn,
    frequence=METEOFRANCE_FREQUENCE)[LISTE_VARIABLES_METEO]

/home/atantet/.conda/envs/agri_notebooks/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
df_meteo_ref = geo.interpolation_inverse_distance_carre(
    df_meteo, df_liste_stations_nn['distance'])

In [8]:
from etp import calcul_etp

etp = calcul_etp(df_meteo_ref, *REF_STATION_LATLON, REF_STATION_ALTITUDE)
df_meteo_ref['etp'] = etp

In [9]:
# Calcul des valeurs journalières des variables météo
s_meteo_ref_jour = pd.Series(dtype=float)
for variable, series in df_meteo_ref.items():
    s_meteo_ref_jour.loc[variable] = VARIABLE_AGGREGATORS[variable](df_meteo_ref[variable])
s_meteo_ref_jour

ff           3.533447e+00
t            2.731364e+02
u            8.991283e+01
ray_glo01    1.467000e+06
rr1          7.106477e+00
etp          3.316577e-01
dtype: float64

In [10]:
# RFU finale cible (mm)
RFU_CIBLE = None

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.1

# Choix de la texture
TEXTURE = 'Terres limoneuses'

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_REMPLIE = 1.

In [11]:
# Choix de la culture
CULTURE = 'Pomme de terre'

# Choix du stade
STADE = 'Vegetation'

In [12]:
import bilan

s_bilan = bilan.calcul_bilan(
    TEXTURE, FRACTION_CAILLOUX,
    CULTURE, STADE,
    s_meteo_ref_jour,
    rfu_cible=RFU_CIBLE, fraction_remplie=FRACTION_REMPLIE, ru_vers_rfu=RU_VERS_RFU)

print(f"RFU initiale pour une profondeur d'enracinement de "
      f"{s_bilan.loc['profondeur_enracinement']:.0f} cm : "
      f"{s_bilan['rfu']:.0f} mm")
print(f"Precipitation : {s_bilan['precipitation']:.2f} mm")
print(f"Évapotranspiration maximale pour la {CULTURE} au stade {STADE} : "
      f"{s_bilan['etm_culture']:.2f} mm")
print(f"Besoin en irrigation pour la {CULTURE} au stade {STADE} "
      f"pour atteindre une rfu_finale de {s_bilan['rfu_cible']:.0f} mm : "
      f"{s_bilan['besoin_irrigation'] :.2f} mm")
if s_bilan['besoin_irrigation'] < 0:
    print("-> Pas besoin d'irriguer")

RFU initiale pour une profondeur d'enracinement de 30 cm : 32 mm
Precipitation : 7.11 mm
Évapotranspiration maximale pour la Pomme de terre au stade Vegetation : 0.30 mm
Besoin en irrigation pour la Pomme de terre au stade Vegetation pour atteindre une rfu_finale de 32 mm : -6.81 mm
-> Pas besoin d'irriguer
